<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Vae_Edditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown ### 4️⃣ Dependencias (No usa gpu, en principio)
!pip install diffusers omegaconf transformers accelerate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=1e8186560bae28348acdb664ce96b801a1704e7b57250ac66fb3f65cd27840b7
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-pyth

In [2]:
#@markdown ### 4️⃣ Descargas el Vae que te guste
import torch
import gdown

# URL del modelo VAE
model_url = "https://civitai.com/api/download/models/83798" #@param {type:"string"}

# Ruta de destino para guardar el archivo .pt
save_path = "vae_modelo.pt"

# Descargar el archivo del modelo VAE desde la URL
gdown.download(model_url, save_path, quiet=False)

# Cargar el modelo VAE desde el archivo .pt
vae = torch.load(save_path)

# Realizar inferencia o modificaciones adicionales con el modelo VAE cargado


Downloading...
From: https://civitai.com/api/download/models/83798
To: /content/vae_modelo.pt
100%|██████████| 335M/335M [00:05<00:00, 59.3MB/s]


In [ ]:
#@markdown ### 4️⃣ Tenes que tranformar ese Vae A Difusser con esto
import argparse
import io
import requests
import torch
from omegaconf import OmegaConf
from diffusers import AutoencoderKL
from diffusers.pipelines.stable_diffusion.convert_from_ckpt import (
    assign_to_checkpoint,
    conv_attn_to_linear,
    create_vae_diffusers_config,
    renew_vae_attention_paths,
    renew_vae_resnet_paths,
)

def custom_convert_ldm_vae_checkpoint(checkpoint, config):
    vae_state_dict = checkpoint
    new_checkpoint = {}
    new_checkpoint["encoder.conv_in.weight"] = vae_state_dict["encoder.conv_in.weight"]
    new_checkpoint["encoder.conv_in.bias"] = vae_state_dict["encoder.conv_in.bias"]
    new_checkpoint["encoder.conv_out.weight"] = vae_state_dict["encoder.conv_out.weight"]
    new_checkpoint["encoder.conv_out.bias"] = vae_state_dict["encoder.conv_out.bias"]
    new_checkpoint["encoder.conv_norm_out.weight"] = vae_state_dict["encoder.norm_out.weight"]
    new_checkpoint["encoder.conv_norm_out.bias"] = vae_state_dict["encoder.norm_out.bias"]
    new_checkpoint["decoder.conv_in.weight"] = vae_state_dict["decoder.conv_in.weight"]
    new_checkpoint["decoder.conv_in.bias"] = vae_state_dict["decoder.conv_in.bias"]
    new_checkpoint["decoder.conv_out.weight"] = vae_state_dict["decoder.conv_out.weight"]
    new_checkpoint["decoder.conv_out.bias"] = vae_state_dict["decoder.conv_out.bias"]
    new_checkpoint["decoder.conv_norm_out.weight"] = vae_state_dict["decoder.norm_out.weight"]
    new_checkpoint["decoder.conv_norm_out.bias"] = vae_state_dict["decoder.norm_out.bias"]
    new_checkpoint["quant_conv.weight"] = vae_state_dict["quant_conv.weight"]
    new_checkpoint["quant_conv.bias"] = vae_state_dict["quant_conv.bias"]
    new_checkpoint["post_quant_conv.weight"] = vae_state_dict["post_quant_conv.weight"]
    new_checkpoint["post_quant_conv.bias"] = vae_state_dict["post_quant_conv.bias"]
    # Retrieves the keys for the encoder down blocks only
    num_down_blocks = len({".".join(layer.split(".")[:3]) for layer in vae_state_dict if "encoder.down" in layer})
    down_blocks = {
        layer_id: [key for key in vae_state_dict if f"down.{layer_id}" in key] for layer_id in range(num_down_blocks)
    }
    # Retrieves the keys for the decoder up blocks only
    num_up_blocks = len({".".join(layer.split(".")[:3]) for layer in vae_state_dict if "decoder.up" in layer})
    up_blocks = {
        layer_id: [key for key in vae_state_dict if f"up.{layer_id}" in key] for layer_id in range(num_up_blocks)
    }
    for i in range(num_down_blocks):
        resnets = [key for key in down_blocks[i] if f"down.{i}" in key and f"down.{i}.downsample" not in key]
        if f"encoder.down.{i}.downsample.conv.weight" in vae_state_dict:
            new_checkpoint[f"encoder.down_blocks.{i}.downsamplers.0.conv.weight"] = vae_state_dict.pop(
                f"encoder.down.{i}.downsample.conv.weight"
            )
            new_checkpoint[f"encoder.down_blocks.{i}.downsamplers.0.conv.bias"] = vae_state_dict.pop(
                f"encoder.down.{i}.downsample.conv.bias"
            )
        paths = renew_vae_resnet_paths(resnets)
        meta_path = {"old": f"down.{i}.block", "new": f"down_blocks.{i}.resnets"}
        assign_to_checkpoint(paths, new_checkpoint, vae_state_dict, additional_replacements=[meta_path], config=config)
    mid_resnets = [key for key in vae_state_dict if "encoder.mid.block" in key]
    num_mid_res_blocks = 2
    for i in range(1, num_mid_res_blocks + 1):
        resnets = [key for key in mid_resnets if f"encoder.mid.block_{i}" in key]
        paths = renew_vae_resnet_paths(resnets)
        meta_path = {"old": f"mid.block_{i}", "new": f"mid_block.resnets.{i - 1}"}
        assign_to_checkpoint(paths, new_checkpoint, vae_state_dict, additional_replacements=[meta_path], config=config)
    mid_attentions = [key for key in vae_state_dict if "encoder.mid.attn" in key]
    paths = renew_vae_attention_paths(mid_attentions)
    meta_path = {"old": "mid.attn_1", "new": "mid_block.attentions.0"}
    assign_to_checkpoint(paths, new_checkpoint, vae_state_dict, additional_replacements=[meta_path], config=config)
    conv_attn_to_linear(new_checkpoint)
    for i in range(num_up_blocks):
        block_id = num_up_blocks - 1 - i
        resnets = [
            key for key in up_blocks[block_id] if f"up.{block_id}" in key and f"up.{block_id}.upsample" not in key
        ]
        if f"decoder.up.{block_id}.upsample.conv.weight" in vae_state_dict:
            new_checkpoint[f"decoder.up_blocks.{i}.upsamplers.0.conv.weight"] = vae_state_dict[
                f"decoder.up.{block_id}.upsample.conv.weight"
            ]
            new_checkpoint[f"decoder.up_blocks.{i}.upsamplers.0.conv.bias"] = vae_state_dict[
                f"decoder.up.{block_id}.upsample.conv.bias"
            ]
        paths = renew_vae_resnet_paths(resnets)
        meta_path = {"old": f"up.{block_id}.block", "new": f"up_blocks.{i}.resnets"}
        assign_to_checkpoint(paths, new_checkpoint, vae_state_dict, additional_replacements=[meta_path], config=config)
    mid_resnets = [key for key in vae_state_dict if "decoder.mid.block" in key]
    num_mid_res_blocks = 2
    for i in range(1, num_mid_res_blocks + 1):
        resnets = [key for key in mid_resnets if f"decoder.mid.block_{i}" in key]
        paths = renew_vae_resnet_paths(resnets)
        meta_path = {"old": f"mid.block_{i}", "new": f"mid_block.resnets.{i - 1}"}
        assign_to_checkpoint(paths, new_checkpoint, vae_state_dict, additional_replacements=[meta_path], config=config)
    mid_attentions = [key for key in vae_state_dict if "decoder.mid.attn" in key]
    paths = renew_vae_attention_paths(mid_attentions)
    meta_path = {"old": "mid.attn_1", "new": "mid_block.attentions.0"}
    assign_to_checkpoint(paths, new_checkpoint, vae_state_dict, additional_replacements=[meta_path], config=config)
    conv_attn_to_linear(new_checkpoint)
    return new_checkpoint
def vae_pt_to_vae_diffuser(
    checkpoint_path: str,
    output_path: str,
):
    # Only support V1
    r = requests.get(
        " https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml"
    )
    io_obj = io.BytesIO(r.content)
    original_config = OmegaConf.load(io_obj)
    image_size = 512
    device = "cuda" if torch.cuda.is_available() else "cpu"

    if checkpoint_path.endswith("safetensors"):
        from safetensors import safe_open

        checkpoint = {}
        with safe_open(checkpoint_path, framework="pt", device="cpu") as f:
            for key in f.keys():
                checkpoint[key] = f.get_tensor(key)
    else:
        checkpoint = torch.load(checkpoint_path, map_location=device)["state_dict"]

    # Convert the VAE model.
    vae_config = create_vae_diffusers_config(original_config, image_size=image_size)
    converted_vae_checkpoint = custom_convert_ldm_vae_checkpoint(checkpoint, vae_config)

    vae = AutoencoderKL(**vae_config)
    vae.load_state_dict(converted_vae_checkpoint)
    vae.save_pretrained(output_path)

vae_pt_path =  '/content/vae_modelo.pt'
dump_path =  '/content/modelo_diffuser.pt'

vae_pt_to_vae_diffuser(vae_pt_path, dump_path)

In [ ]:
#@markdown ### 4️⃣  Y aca es donde comienza tu creatividad
import os
import torch
from diffusers import AutoencoderKL
from torch import nn

# Definir los valores de los parámetros
model_path = '/content/modelo_diffuser.pt'  # Ruta del modelo en Google Drive
output_path = '/content/Finichio.pt'  # Ruta de salida del modelo modificado en Google Drive
contrast = 0.5  # @param {type:"number"}
#@markdown 'mul' para multiplicar, 'add' para agregar , si no pones esto salta error, asi que no la vas a cagar
contrast_operation = 'mul'  #@param {type:"string"}
brightness_operation = 'add'  #@param {type:"string"}
brightness = 0  #@param {type:"number"}
patch_encoder = True  #@param {type:"boolean"}

print('Loading model...')
vae = AutoencoderKL.from_pretrained(model_path, subfolder='vae', latent_channels=4)

print('Applying modifications...')
if contrast_operation == 'add':
    vae.decoder.conv_out.weight = nn.Parameter(vae.decoder.conv_out.weight + contrast)
    if patch_encoder:
        vae.encoder.conv_in.weight = nn.Parameter(vae.encoder.conv_in.weight - contrast)
elif contrast_operation == 'mul':
    vae.decoder.conv_out.weight = nn.Parameter(vae.decoder.conv_out.weight * contrast)
    if patch_encoder:
        vae.encoder.conv_in.weight = nn.Parameter(vae.encoder.conv_in.weight / contrast)
else:
    raise ValueError('Invalid contrast operation')

if brightness_operation == 'add':
    vae.decoder.conv_out.bias = nn.Parameter(vae.decoder.conv_out.bias + brightness)
    if patch_encoder:
        vae.encoder.conv_in.bias = nn.Parameter(vae.encoder.conv_in.bias - brightness)
elif brightness_operation == 'mul':
    vae.decoder.conv_out.bias = nn.Parameter(vae.decoder.conv_out.bias * brightness)
    if patch_encoder:
        vae.encoder.conv_in.bias = nn.Parameter(vae.encoder.conv_in.bias / brightness)
else:
    raise ValueError('Invalid brightness operation')

print('Saving...')
torch.save(vae.state_dict(), output_path)

print('Done!')


In [ ]:
#@markdown ### 5️⃣ Descarga
import os


#@markdown Ruta del directorio de imágenes Ruta del directorio de imágenes
directorio_imagenes = '/content/Finichio.pt'

# Descarga el archivo zip
from google.colab import files
files.download(directorio_imagenes )

In [ ]:
#@markdown ### 5️⃣ Verificamos State Dict
import torch

file_path = '/content/Finichio.pt'
# Función para obtener el tamaño del archivo en MB
def get_file_size(file_path):
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)
    return size_mb
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_0_size = get_file_size(file_path)
print(f"Tamaño del archivo: {model_0_size} MB")
try:
    state_dict = torch.load(file_path, map_location=device)
    print("Contenido parcial del state_dict:")
    print("-" * 40)
    print("Primeras claves y formas de tensor:")
    for key, value in list(state_dict.items())[:5]:
        print(f"Clave: {key}")
        print(f"Forma del tensor: {value.shape}")
        print("-" * 20)
except Exception as ex:
    print(f'ERROR al cargar el archivo .ckpt: {ex}')


In [18]:
#@markdown ### 5️⃣ Mas informacion hacerca la parte importante
print('Esta poronga se respala en la siguiente Documentacion: https://github.com/sALTaccount/VAE-BlessUp/blob/master/README.md')


Esta poronga se respala en la siguiente Documentacion: https://github.com/sALTaccount/VAE-BlessUp/blob/master/README.md
